# Import packages

In [39]:
import numpy as np
import scipy.sparse.linalg as lg
import time
from copy import deepcopy

# PQR

In [2]:
def PQR(matrix, n, m, r = 0, eps = 0):

    if not r or r > min(m, n):
        r = min(m, n)

    Q = np.zeros((n, r))
    R = np.zeros((r, m))
    P = np.arange(m)

    tol = eps + 1
    i = 0

    while i < r and tol > eps:

        norms = np.array([np.linalg.norm(matrix[:, j]) for j in range(i, m)])
        tol = np.sqrt(np.sum(norms**2))
        args = np.argsort(norms)[::-1]
        args += i

        P[[i, args[0]]] = P[[args[0], i]]
        R[:, [i, args[0]]]  = R[:, [args[0], i]]
        matrix[:, [i, args[0]]] = matrix[:, [args[0], i]]

        R[i, i] = norms[args[0] - i]
        Q[:, i] = matrix[:, i] / R[i, i]

        for j in range(i + 1, m):
            R[i, j] = matrix[:, j] @ Q[:, i]
            matrix[:, j] -= R[i, j] * Q[:, i]

        i += 1

    return Q, R, P, i

# MaxVol

In [704]:
def maxvol(mat, n, m, r, eps = 0):

    if not r or r > min(n, m):
          r = min(n, m)

    rows = np.random.permutation(n)
    B = mat[rows[:r], :]
    _, _, cols, rank = PQR(B, r, m, r, eps)

    rev_rows = np.zeros(n, dtype=int)
    rev_cols = np.zeros(m, dtype=int)

    for i in range(n):
        rev_rows[rows[i]] = i
    for i in range(m):
        rev_cols[cols[i]] = i

    matrix = mat[np.ix_(rows, cols)]

    R = np.arange(r)

    C = matrix[:, :r]
    A = C[:r, :]

    inv_A = np.linalg.pinv(A)

    prod = C @ inv_A

    max_val_ind = np.unravel_index(np.argmax(np.abs(prod), axis=None), prod.shape)

    iter = 0

    while np.abs(prod[max_val_ind]) > 1.000001 and max_val_ind[0] >= r:

        iter += 1

        vec = prod[max_val_ind[0], :].copy()
        vec[max_val_ind[1]] -= 1
        vec /= prod[max_val_ind]

        prod -= prod[:, max_val_ind[1]].reshape((-1, 1)) @ vec.reshape((1, -1))

        R[max_val_ind[1]] = max_val_ind[0]

        max_val_ind = np.unravel_index(np.argmax(np.abs(prod), axis=None), prod.shape)

    prod = prod[rev_rows, :]
    R = matrix[np.ix_(R, rev_cols)]

    return prod, R

# Cross Approximation

In [4]:
def cross_approx(matrix, n, m, r = 0, eps = 0):

    if not r and r > min(m, n):
        r = min(m, n)

    Q = np.zeros((n, r))
    R = np.zeros((r, m))

    vec = np.arange(m)

    i = 0

    max_el = eps + 1

    while i < r and max_el * np.sqrt((n - i) * (m - i)) > eps:

        ind = np.random.randint(m - i)
        col = vec[ind]

        row = np.argmax(np.abs(matrix[:, col] - Q[:, :i] @ R[:i, col]), axis=None)
        col = np.argmax(np.abs(matrix[row, :] - Q[row, :i] @ R[:i, :]), axis=None)

        val_col = matrix[:, col] - Q[:, :i] @ R[:i, col]
        row = np.argmax(np.abs(val_col), axis=None)

        val_row = matrix[row, :] - Q[row, :i] @ R[:i, :]

        Q[:, i] = val_col / val_col[row]
        R[i, :] = val_row

        max_el = np.abs(val_col[row])

        ind = np.where(vec == col)[0][0]

        vec = np.delete(vec, ind)

        i += 1

    return Q, R, i

# Cross Approximation Dzheltkov

In [664]:
def cross_approx_dzh(matrix, n, m, r = 0, eps = 0):

    if not r and r > min(m, n):
        r = min(m, n)

    Q = np.zeros((n, r))
    R = np.zeros((r, m))

    i = 0
    err = eps + 1

    cols = np.zeros((n, r + 1))
    ind_cols = np.ones(r + 1, dtype=int) * m
    ind_cols[0] = np.random.randint(m)
    cols[:, 0] = matrix[:, ind_cols[0]]
    free_col = 1

    vec_cols = np.arange(m)

    rows = np.zeros((r + 1, m))
    ind_rows = np.ones(r + 1, dtype=int) * n
    ind_rows[0] = np.argmax(np.abs(cols[:, 0]))
    rows[0, :] = matrix[ind_rows[0], :]
    free_row = 1

    vec_rows = np.arange(n)

    while i < r and err > eps:

        row_1, col_1 = np.unravel_index(np.argmax(np.abs(cols[:, :free_col])), cols[:, :free_col].shape)
        row_2, col_2 = np.unravel_index(np.argmax(np.abs(rows[:free_row, :])), rows[:free_row, :].shape)

        if row_1 == ind_rows[row_2] and col_2 == ind_cols[col_1]:

            Q[:, i] = cols[:, col_1] / cols[row_1, col_1]
            R[i, :] = rows[row_2, :]

            cols = np.delete(cols, col_1, 1)
            rows = np.delete(rows, row_2, 0)

            ind_cols = np.delete(ind_cols, col_1)
            ind_rows = np.delete(ind_rows, row_2)

            free_col -= 1
            free_row -= 1

            cols[:, :free_col] -= Q[:, i].reshape((-1, 1)) @ R[i, ind_cols[:free_col]].reshape((1, -1))
            rows[:free_row, :] -= Q[ind_rows[:free_row], i].reshape((-1, 1)) @ R[i, :].reshape((1, -1))

            ind = np.where(vec_cols == col_2)[0][0]
            vec_cols = np.delete(vec_cols, ind)

            ind = np.where(vec_rows == row_1)[0][0]
            vec_rows = np.delete(vec_rows, ind)

        elif np.abs(cols[row_1, col_1]) > np.abs(rows[row_2, col_2]):

            rows[free_row, :] = matrix[row_1, :] - Q[row_1, :i] @ R[:i, :]
            col = np.argmax(np.abs(rows[free_row, :]))

            if col == ind_cols[col_1]:

                Q[:, i] = cols[:, col_1] / cols[row_1, col_1]
                R[i, :] = rows[free_row, :]

                cols = np.delete(cols, col_1, 1)
                ind_cols = np.delete(ind_cols, col_1)
                free_col -= 1

                ind = np.where(vec_rows == row_1)[0][0]

            else:

                ind_rows[free_row] = row_1
                free_row += 1

                Q[:, i] = matrix[:, col] - Q[:, :i] @ R[:i, col]
                row = np.argmax(np.abs(Q[:, i]))

                R[i, :] =  (matrix[row, :] - Q[row, :i] @ R[:i, :]) / Q[row, i]

                ind = np.where(vec_rows == row)[0][0]

            vec_rows = np.delete(vec_rows, ind)

            cols[:, :free_col] -= Q[:, i].reshape((-1, 1)) @ R[i, ind_cols[:free_col]].reshape((1, -1))
            rows[:free_row, :] -= Q[ind_rows[:free_row], i].reshape((-1, 1)) @ R[i, :].reshape((1, -1))

            ind = np.where(vec_cols == col)[0][0]
            vec_cols = np.delete(vec_cols, ind)

        else:

            cols[:, free_col] = matrix[:, col_2] - Q[:, :i] @ R[:i, col_2]
            row = np.argmax(np.abs(cols[:, free_col]))

            if row == ind_rows[row_2]:

                R[i, :] = rows[row_2, :] / rows[row_2, col_2]
                Q[:, i] = cols[:, free_col]

                rows = np.delete(rows, row_2, 0)
                ind_rows = np.delete(ind_rows, row_2)
                free_row -= 1

                ind = np.where(vec_cols == col_2)[0][0]

            else:

                ind_cols[free_col] = col_2
                free_col += 1

                R[i, :] = matrix[row, :] - Q[row, :i] @ R[:i, :]
                col = np.argmax(np.abs(R[i, :]))

                Q[:, i] =  (matrix[:, col] - Q[:, :i] @ R[:i, col]) / R[i, col]

                ind = np.where(vec_cols == col)[0][0]

            vec_cols = np.delete(vec_cols, ind)

            cols[:, :free_col] -= Q[:, i].reshape((-1, 1)) @ R[i, ind_cols[:free_col]].reshape((1, -1))
            rows[:free_row, :] -= Q[ind_rows[:free_row], i].reshape((-1, 1)) @ R[i, :].reshape((1, -1))

            ind = np.where(vec_rows == row)[0][0]
            vec_rows = np.delete(vec_rows, ind)

        i += 1

        if not free_col:
            ind_cols[0] = vec_cols[np.random.randint(m - i)]
            cols[:, 0] = matrix[:, ind_cols[0]] - Q[:, :i] @ R[:i, ind_cols[0]]
            free_col = 1

        if not free_row:
            ind_rows[0] = vec_rows[np.random.randint(n - i)]
            rows[0, :] = matrix[ind_rows[0], :] - Q[ind_rows[0], :i] @ R[:i, :]
            free_row = 1

        err = np.linalg.norm(rows[:free_row, :])**2 + np.linalg.norm(cols[:, :free_col])**2 - np.linalg.norm(rows[:free_row, ind_cols[:free_col]])**2

        err = np.sqrt(err / (n * free_col + m * free_row - free_col * free_row) * (n - i) * (m - i))

    return Q, R, i

# R-SVD

In [107]:
def rsvd(matrix, n, m, r):

    flag = 1

    if n > m:
        matrix = matrix.T
        n, m = m, n
        flag = 0

    N = np.random.rand(m, r * max(1, int(np.log10(m))))

    Q, R = np.linalg.qr(matrix @ N)

    U, s, V = lg.svds(Q.T @ matrix, k = r)

    U = Q @ U

    if flag:
        return U, s, V
    else:
        return V.T, s, U.T

# Create matrix

In [108]:
n = 1000
m = 1000
mat_cr = np.zeros((n, m))

for i in range(n):
    for j in range(m):
        mat_cr[i, j] = 1 / (i + j + 1)



In [113]:
matrix = mat_cr.copy()
mat_copy = mat_cr.copy()

# Run R-SVD

In [114]:
start_time = time.time()
U, s, V = rsvd(matrix, n, m, 24)
end_time = time.time()
print(f"time = {end_time - start_time}")

print(np.linalg.norm((U * s) @ V - mat_copy) / np.linalg.norm(mat_copy))

time = 0.15114521980285645
8.587844140094899e-13


# Run Cross Approximation

In [672]:
start_time = time.time()
Q, R, rank = cross_approx(matrix, n, m, 30, eps = 10**(-10))
end_time = time.time()
print(f"time = {end_time - start_time}")

print(np.linalg.norm(Q @ R - mat_copy) / np.linalg.norm(mat_copy))
print(f'rank = {rank}')

time = 0.031072378158569336
4.597909895129225e-13
rank = 24


# Run Cross Approximation Dzheltkov

In [674]:
start_time = time.time()
Q, R, rank = cross_approx_dzh(matrix, n, m, 30, eps = 10**(-10))
end_time = time.time()
print(f"time = {end_time - start_time}")

print(np.linalg.norm(Q @ R - mat_copy) / np.linalg.norm(mat_copy))
print(f'rank = {rank}')

time = 0.011477947235107422
5.944791153644091e-11
rank = 21


# Run PQR

In [680]:
start_time = time.time()
Q, R, P, rank = PQR(matrix, n, m, 30, eps = 10**(-10))
end_time = time.time()
print(f"time = {end_time - start_time}")

print(np.linalg.norm(Q @ R - mat_copy[:, P]) / np.linalg.norm(mat_copy))
print(f'rank = {rank}')

time = 0.378800630569458
2.41639545483056e-12
rank = 22


# Run MaxVol

In [728]:
start_time = time.time()
Q, R = maxvol(matrix, n, m, 22)
end_time = time.time()
print(f"time = {end_time - start_time}")

print(np.linalg.norm(Q @ R - mat_copy) / np.linalg.norm(mat_copy))

time = 0.2122347354888916
8.25962852270186e-05
